# Acoustic localization using linearized inversion

## Purpose of this notebook
The purpose of this notebook is to show all the processing steps and python commands needed to localize an acoustic signal in three-dimensions (3D) using linearized inversion. The example below uses data from the large audio-video array and corresponds to the localization results presented in figure 9 of the paper. Note that this notebook is only meant to be show the analysis process on a single example. For processing, a full deployment efficiently, the code should be slightly modified (i.e., removing all the plotting steps and looping through files automatically) and executed in a regular python file (.py). Details about the theory behind the detection and localization process can be found in the paper.

The analysis process is composed of 6 steps:
* Step 1: Importing all the necessary libraries
* Step 2: Defining and loading the configuration files
* Step 3: Finding data from all hydrophones
* Step 4: Running the automatic detector
* Step 5: Localizing the detected signals
* Step 6: Saving the localization results

All the data and config files needed for this notebook are in the folder [./localization/large-array](https://github.com/xaviermouy/XAV-arrays/tree/main/localization/large-array).

## Step1: Importing all the necessary libraries
The libraries used for the processing include ecosound, pandas and matplotlib (for plotting). All scripts used for the automatic detection and localization steps are in the python files detection.py and localization.py, respectively. The python file in tools.py contains other non-specific functions used throughout the processing.

Before going through the notebook, you will need to install the library ecosound and ipympl. To do so, type the following:

In [ ]:
!pip install ecosound --upgrade
!pip install ipympl

If you run this notebook on Google Colab, you will need to execute the commands above at the beginning of each session. If you run this notebook localy on your own machine, **you only need to do this once**. Once all the libraries are installed, you can import them:

In [1]:
%matplotlib widget 
import pandas as pd
import ecosound.core.tools
from ecosound.core.metadata import DeploymentInfo
import ecosound
import tools
import detection
import localization
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ecosound'

## Step 2: Defining and loading the configuration files

Before starting the processing we need to define four configuration files that define some metadata, describe the configuration of the array, and define the parameters to use for the detection and localization. Once these configuration files are defined, they can be used for processing the entire deployment.

### Deployment metadata 
The first configration file is the deployment_info.csv. It defines the basic metadata about the deployment (recorder type, depths, coordinates, etc.). These metadata will be attached to the detection and localization results. Each line of the csv file define a separate channel/hydrophone. The field names should be relatively self-explanatory. To see the format to use, have a look at the example of deployment_info files used for the large array ([here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/large-array/deployment_info.csv)) and mobile array ([here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/mobile-array/deployment_info.csv)).

You need to define which deployment_info file to use,then you can load it and inspect that all the fields are filled correctly:

In [ ]:
# config file
deployment_info_file = r'.\large-array\deployment_info.csv'

# load and display deployment metadata
Deployment = DeploymentInfo()
Deployment.read(deployment_info_file)
Deployment.data

### Hydrophones configuration

The second configration file is the hydrophones_config csv file. It defines for each hydrophone, the coordinates, the channel the data is recorded to, and the location (path) of the data. See the examples of hydrophones_config file for the large array ([here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/large-array/hydrophones_config_07-HI.csv)) and the mobile array ([here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/mobile-array/hydrophones_config_HI-201909.csv)). Note that the first channel starts at 0.

Let's define and load the hydrophones_config file for the large array:

In [ ]:
# Define config file
hydrophones_config_file = r'.\large-array\hydrophones_config_07-HI.csv'

# load and plot hydrophone configuration
hydrophones_config= pd.read_csv(hydrophones_config_file, skipinitialspace=True, dtype={'name': str, 'file_name_root': str}) # load hydrophone coordinates (meters)
hydrophones_config

We can see that there is one row per hydrophone and 8 fields per row: 

* The **name** is a free field only used to label hydrophones in plots.
* The **x**, **y**, and **z** define the 3D coordinates of the hydrophones in meters. 
* The **array_channel** identifies which channel the hydrophone corresponds to. It must be consistent with the **audio_channel_number** values identified in the deployment_info file. 
* The **audio_file_channel** identifies which channel is used in the audio file. Some recorders (like the AMAR recorders) save data from each channel in separate mono wav files. In such case, the **audio_file_channel** is set to 0 (since each file has only 1 channel). Other recorders (like the Soundtrap recorders) save data from all channels in a single audio file. In such case, the channel to use in the audio file must be identified. In the present case, we use data from the large array, which save data from each channel in a seperate mono file. So we set the "audio_file_channel" to 0 for all hydrophones. 
* **file_name_root** defines the first part of the filenames for each hydrophone. It is assumed that the remaining part of the audio file contains the time stamp of the file. For data from AMAR recorders, the audio file names start with the serial number of the AMAR, a dot, the channel number, a dot, and the timestamp. So, the **file_name_root** is different for each hydrophone. For Soundtraps recorders, data from all hydrophones are saved in a single multi-channel file, so **file_name_root** is the same for all hydrophones.

To ensure that the hydrophone geometry makes sense, we can plot the hydrophoen locations:

In [ ]:
fig, ax = localization.plot_localizations3D(hydrophones=hydrophones_config)

### Detection parameters
The third configuration file to define is the detection file. It is a .yaml files defining the different parameters used during the detection process. It includes the spectrogram, denoising and detections parameters. A description of each parameter can be found in the paper. An example of detection config file can be found [here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/large-array/detection_config_large_array.yaml).

Let's define and load the detection config file:

In [ ]:
# Define config files
detection_config_file = r'.\large-array\detection_config_large_array.yaml'

# load configuration parameters
detection_config = ecosound.core.tools.read_yaml(detection_config_file)
detection_config

### localization parameters

The last configuration file is the localization file (.yaml) that defines the different parameters for the linearized inversion. An example of localization config file can be found [here](https://github.com/xaviermouy/XAV-arrays/blob/main/localization/large-array/localization_config_large_array.yaml).

Let's define and load the localization config file:

In [ ]:
localization_config_file = r'.\large-array\localization_config_large_array.yaml'
localization_config = ecosound.core.tools.read_yaml(localization_config_file)
localization_config

Each parameter is explained below:
* **sound_speed_mps:** sound speed in the water, in meters per second
* **ref_channel:** reference channel for the TDOA measurements
* **upsample_res_sec:** time resolution desired for the cross correlation (in seconds)
* **min_corr_val:**  minimum correlation values accepted for considering a TDOA measurement valid
* **start_model:** coordinates of the first starting model to use in the linearized inversion (typically at teh center of the array, i.e. [0,0,0]).
* **start_model_repeats:** number of times the linearized inversion is repeated using different starting models.
* **damping_factor:** damping factor to use between iterations of the linearized inversion.
* **stop_delta_m:** stoping criteria. Stops when the difference of model norms between two consecutive iterations is less than stop_delta_m.
* **stop_max_iteration:** stoping criteria. Maximum number of iterations allowed.

## Step 3: Finding data from all hydrophones

We can now define the audio file we want to process. We can specify the name of a file from one hydrophone, and the code below will find out the corresponding files from the other hydrophones.

In [ ]:
# Define file  to process
infile = r'.\large-array\data\AMAR173.4.20190920T163858Z.wav'

# Look up data files for all channels
audio_files = tools.find_audio_files(infile, hydrophones_config)

The variable **audio_files** now lists the path and name of the audio files from each hydrophone.

In [ ]:
audio_files

We can then inspect the data from all channels by displaying the spectrograms and waveforms. The spectrogram parameters used for the spectrograms are the ones defined in the detection_config file defined earlier.

In [ ]:
# plot all data 
graph_spectros, graph_waveforms = tools.plot_all_channels(audio_files,
                                                          detection_config['SPECTROGRAM']['frame_sec'],
                                                          detection_config['SPECTROGRAM']['window_type'],
                                                          detection_config['SPECTROGRAM']['nfft_sec'],
                                                          detection_config['SPECTROGRAM']['step_sec'],
                                                          detection_config['SPECTROGRAM']['fmin_hz'],
                                                          detection_config['SPECTROGRAM']['fmax_hz'],
                                                          detections_channel=detection_config['AUDIO']['channel'],
                                                          verbose=False)

#graph_spectros.show(display=False)
fig_graph_waveforms, ax_graph_waveforms = graph_waveforms.show(display=False)
fig_graph_spectros, ax_graph_spectros = graph_spectros.show(display=False)

Stacked spectrograms for all hydrophones:

In [ ]:
fig_graph_spectros

Stacked waveforms for all hydrophones:

In [ ]:
fig_graph_waveforms

## Step 4: Running the automatic detector

Teh detector can now be run on one of the hydrophones using the function **detection.run_detector**. The input arguments for that function are (in this order):
1. The path of the file to process, 
2. The channel to use,
3. the detection_config parameters (see step 2),
4. the path of the deployment_info file (see step 2). 

In [ ]:
# run detector on selected channel
detections = detection.run_detector(audio_files['path'][detection_config['AUDIO']['channel']],
                                    audio_files['channel'][detection_config['AUDIO']['channel']],
                                    detection_config,
                                    deployment_file=deployment_info_file)

The detector returns the variable **detections**, which is an ecosound annotation object. All detections can be founn by accessing **detections.data**. The start and stop times of each detection are defined by the fields **time_min_offset** and **time_max_offset**, respectively. The minimum and maximum frequency of the detection boxes are defined by the fields **frequency_min** and **frequency_min**, respectively.

Here, we see that the algorithm found 34 detections:

In [ ]:
# Look at detection table
detections.data

We can inspect the detections by ploting their time and frequency boxes on top of the spectrogram and waveform:

In [ ]:
# plot detections
graph_detec = tools.plot_single_channel(audio_files['path'][detection_config['AUDIO']['channel']],
                                  detection_config['SPECTROGRAM']['frame_sec'],
                                  detection_config['SPECTROGRAM']['window_type'],
                                  detection_config['SPECTROGRAM']['nfft_sec'],
                                  detection_config['SPECTROGRAM']['step_sec'],
                                  detection_config['SPECTROGRAM']['fmin_hz'],
                                  detection_config['SPECTROGRAM']['fmax_hz'],
                                  detections=detections,
                                  verbose=False)
fig_graph_detec, ax_graph_detec = graph_detec.show(display=False)
fig_graph_detec

## Step 5: Localizing the detected signals

Now we can localize the detected signals using linearized inversion with data from all hydrophones. The code below may take several seconds.

In [ ]:
# Perform localization using linearized invertion
localizations = localization.LinearizedInversion.run_localization(audio_files, detections, deployment_info_file, detection_config, hydrophones_config, localization_config, verbose=False)

The localization results are in the **localizations** variable. It contains the same number of rows as the variable **detections**. Localization details can be found in **localizations.data**, and the localization fileds are as follows:
* **x**, **y**, and **z** define the coordinates of the localizations
* **x_err_low** and **x_err_high**, **y_err_low** and **y_err_high**,and **z_err_low** and **z_err_high** are the low and high localization uncertainties for each dimension.
* **x_err_span**, **y_err_span**, and **z_err_span** define the span of the uncertainties in each dimension (i.e., x_err_span = x_err_high - x_err_low).

In [ ]:
localizations.data

Note that some rows (i.e., detections) could not be localized (i.e., localization fields = NaNs) because the correlation value during the TDOA measurement process was too low, or because the linearized inversion process did not converge. There are also localizations that have large uncertainties that would not be very useful for matching with the video data.

So, we can filter the localizations so we only keep the ones with an uncertainty span less than 1 m:

In [ ]:
# Show that some detections with no localizations (nan) -> did not converge

# Filter localization results to only keep results with low uncertainty
localizations.filter("x_err_span < 1 & y_err_span < 1 & z_err_span < 1.5", inplace=True)



When we plot the signals that are remaining (after filtering the localizations with large uncertainties), we see that it kept all the fish sounds:

In [ ]:
# plot spectrogram with measurements filtered
graph_loc = tools.plot_single_channel(audio_files['path'][detection_config['AUDIO']['channel']],
                                      detection_config['SPECTROGRAM']['frame_sec'],
                                      detection_config['SPECTROGRAM']['window_type'],
                                      detection_config['SPECTROGRAM']['nfft_sec'],
                                      detection_config['SPECTROGRAM']['step_sec'],
                                      detection_config['SPECTROGRAM']['fmin_hz'],
                                      detection_config['SPECTROGRAM']['fmax_hz'],
                                      detections=localizations,
                                      verbose=False)
fig_graph_loc, ax_graph_loc = graph_loc.show(display=False)
fig_graph_loc

Finally, we can visualize the localizations (blue dots) with their unceratinties (red bars) in a 3D scatter plot:

In [ ]:
# Plot localizations
fig_loc, fig_ax = localization.plot_localizations3D(localizations=localizations, hydrophones=hydrophones_config)
fig_loc

## Step 6: Saving the localization results
The loclaization results can be saved to a csv file and/or a netcdf file that can be used later on by ecosound:

In [ ]:
# save as csv:
localizations.to_csv('localization_results.csv')

# save as netcdf
localizations.data.drop(columns=['iterations_logs'],inplace=True)
localizations.to_netcdf('localization_results.nc')